# Fusion des brevets avec leur classification

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
dataroot = "/content/drive/My Drive/EISTI 2021/Données"

In [ ]:
with open("/content/drive/Shareddrives/ING3 IA: Use Case 1 (NLP Patent)/classification_36_reduce.txt",'r') as f:
    contents = f.read()
espace = contents[1]
classification = contents.replace(espace, "")
classification_split = classification.split("\n\n")

In [ ]:
publi_number = []
publi_kind = []
publi_date = []
publi_classification = []
publi_information = []
for l in classification_split:
    if len(l) >= 1 and l[0] == "\"":
        tmp_classification = []
        tmp_information = []
        publi_number.append(l.split(";")[0].split(" ")[1])
        publi_kind.append(l.split(";")[0].split(" ")[2])
        publi_date.append(l.split(";")[0].split(" ")[3][:-1])
        tmp_classification.append(l.split(";")[1].split(" ")[0] + " " + l.split(";")[1].split(" ")[1])
        try:
            tmp_information.append(l.split(";")[1].split(" ")[2])
        except:
            tmp_information.append("")
    elif len(l) >= 1:
        tmp_classification.append(l.split(" ")[0] + " " + l.split(" ")[1])
        try:
            tmp_information.append(l.split(" ")[2])
        except:
            tmp_information.append("")
    if len(l) >= 1 and l[-1] == "\"":
        tmp_classification[0] = tmp_classification[0][1:]
        tmp_information[-1] = tmp_information[-1][:-1]
        publi_classification.append(tmp_classification)
        publi_information.append(tmp_information)            

In [ ]:
import pandas as pd

publication = pd.DataFrame()
publication["number"] = publi_number
publication["kind"] = publi_kind
publication["date"] = publi_date
publication["information"] = publi_information
publication["classification"] = publi_classification

In [ ]:
publication_unique = publication.drop_duplicates(["number"], keep='last')
publication_unique.set_index("number", inplace=True)

In [ ]:
contenu_number = []
contenu_language = []
contenu_subject = []
contenu_content = []
with open("/content/drive/My Drive/EISTI 2021/Données/fulltext-EP3600000.txt",'r') as f:
    for l in f:
      split_l = l.split("\t")
      contenu_number.append(split_l[1])
      contenu_language.append(split_l[4])
      contenu_subject.append(split_l[5])
      contenu_content.append(split_l[7])

In [ ]:
contenu = pd.DataFrame()
contenu["number"] = contenu_number
contenu["language"] = contenu_language
contenu["subject"] = contenu_subject
contenu["contenu"] = contenu_content

## Exemple avec l'export d'un fichier en français pour la classification

In [ ]:
revendications = contenu[contenu["subject"] == "CLAIM"].copy()
del revendications["subject"]
revendications.columns = ["number", "language", "claim"]
revendications_fr = revendications[revendications["language"] == "fr"].copy()
revendications_fr = revendications_fr.drop_duplicates(["number"], keep='last')
del revendications_fr["language"]
revendications_fr.set_index("number", inplace=True)

In [ ]:
description = contenu[contenu["subject"] == "DESCR"].copy()
del description["subject"]
description.columns = ["number", "language", "description"]
description_fr = description[description["language"] == "en"].copy()
description_fr = description_fr.drop_duplicates(["number"], keep='last')
del description_fr["language"]
description_fr.set_index("number", inplace=True)

In [ ]:
titles = contenu[contenu["subject"] == "TITLE"].copy()
del titles["subject"]
titles.columns = ["number", "language", "title"]
titles_fr = titles[titles["language"] == "en"].copy()
titles_fr = titles_fr.drop_duplicates(["number"], keep='last')
del titles_fr["language"]
titles_fr.set_index("number", inplace=True)

In [ ]:
abstr = contenu[contenu["subject"] == "ABSTR"].copy()
del abstr["subject"]
abstr.columns = ["number", "language", "abstr"]
abstr_fr = abstr[abstr["language"] == "en"].copy()
abstr_fr = abstr_fr.drop_duplicates(["number"], keep='last')
del abstr_fr["language"]
abstr_fr.set_index("number", inplace=True)

In [ ]:
jointure = publication_unique.join(revendications_fr)
jointure = jointure.join(description_fr)
jointure = jointure.join(titles_fr)
jointure = jointure.join(abstr_fr)

In [ ]:
jointure.dropna(how="any", inplace=True)

In [ ]:
jointure.iloc[:10000].to_csv("/content/drive/Shareddrives/ING3 IA: Use Case 1 (NLP Patent)/data_clean/35_en_reduce_publications.csv")

## Exemple avec l'export en français pour la génération, en ne gardant que l'abstract et les revendications

In [ ]:
en_reduce_generation = jointure.copy()
del en_reduce_generation["kind"]
del en_reduce_generation["date"]
del en_reduce_generation["information"]
del en_reduce_generation["classification"]
del en_reduce_generation["description"]
del en_reduce_generation["title"]

In [ ]:
from bs4 import BeautifulSoup
en_reduce_generation["claim"] = [BeautifulSoup(abstr).get_text() for abstr in en_reduce_generation['claim']]
en_reduce_generation['abstr'] = [BeautifulSoup(abstr).get_text() for abstr in en_reduce_generation['abstr']]
en_reduce_generation

In [ ]:
en_reduce_generation.to_csv("/content/drive/Shareddrives/ING3 IA: Use Case 1 (NLP Patent)/data_clean/tmp2_en_claim_abstract.csv")